### 동적으로 움직이는 페이지

구글 인기영화 - 할인 중인 영화

### 페이지 로딩

In [1]:
import time
from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [2]:
!ls

10_bs4_cupang-pages.ipynb
11_daum_movies.ipynb
시가총액1-200.csv
12_csv_stock.ipynb
13_selenium.ipynb
14_selenium-flight.ipynb
15_selenium-movies.ipynb
16_selenium-movies_scroll_scraping.ipynb
17_headless_selenium-movies_scroll_scraping.ipynb
18_headless_selenium-movies_scroll_scraping_user_agent-Copy1.ipynb
18_headless_selenium-movies_scroll_scraping_user_agent.ipynb
1_html.ipynb
2_xpath.ipynb
3_requests.ipynb
4_re.ipynb
5_user_agent.ipynb
6_bs4.ipynb
7_bs4_webtoons.ipynb
8_bs4_gauss.ipynb
9_bs4_cupang.ipynb
chromedriver.exe
chromedriver_linux
google_movie.png
movie.html
nadocoding.html
쿠팡.png
Untitled.ipynb


In [3]:
browser = webdriver.Chrome("chromedriver.exe")
browser.maximize_window() # 최대화

WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [3]:
url = "https://play.google.com/store/movies/top"

In [4]:
browser.get(url)

In [5]:
# 보충 설명 : PC 해상도 확인 방법 : 바탕화면 > 마우스 우클릭 > 디스플레이 설정 메뉴에서 PC 해상도 확인 가능

# 지정한 스크롤 내리기 > 모니터(해상도) 높이인 1080 위치로 스크롤 내리기
#browser.execute_script("window.scrollTo(0,1080)") # 내 PC 해상도 : 1920 x 1080

In [6]:
# 화면 가장 아래로 스크롤 내리기
#browser.execute_script("window.scrollTo(0,document.body.scrollHeight)") # 내 PC 해상도 : 1920 x 1080

In [7]:
# 2초에 한번씩 스크롤 내림
interval = 2 

In [8]:
# 현재 문서 높이를 가져와서 저장
prev_height = browser.execute_script("return document.body.scrollHeight")

In [9]:
# 무한 반복 수행
while True:
    # 스크롤을 가장 아래로 내림
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight)") 
    
    # 페이지 로딩 대기
    time.sleep(interval)
    
    # 현재 문서 뫂이를 가져와서 저장
    curr_height = browser.execute_script("return document.body.scrollHeight")
    if curr_height == prev_height:
        break
        
    prev_height = curr_height

In [10]:
print("스크롤 완료")

스크롤 완료


### 스크래핑

In [11]:
import requests
from bs4 import BeautifulSoup

In [12]:
soup = BeautifulSoup(browser.page_source, "lxml")

In [13]:
movies = soup.find_all("div", attrs={"class":"Vpfmgd"})

# 복수게 Class 는 List 로
#movies = soup.find_all("div", attrs={"class":["ImZGtf mpg5gc","Vpfmgd"]}) 

In [14]:
print(len(movies))

200


In [15]:
# with open("movie.html", "w", encoding="utf-8-sig") as f:
#     #f.write(res.text)
#     f.write(soup.prettify()) # html 문서를 예쁘게 출력

In [16]:
# 할인된 영화 목록 정리
for movie in movies:
    title = movie.find("div", attrs={"class":"WsMG1c nnK0zc"}).get_text()

    # 할인 전 가격
    original_price = movie.find("span", attrs={"class":"SUZt4c djCuy"})
    if original_price:
        original_price = original_price.get_text()
    else:
        #print(title, "<할인되지 않는 영화 제외>")
        continue
        
    # 할인된 가격
    price = movie.find("span", attrs={"class":"VfPpfd ZdBevf i5DZme"}).get_text()
    
    # 링크
    link = movie.find("a", attrs={"class":"JC71ub"})["href"]
    
    # baseURL = "https://play.google.com"
    
    print(f"제목 : {title}")
    print(f"할인 전 가격 : {original_price}")
    print(f"할인 가격 : {price}")
    print(f"링크 : " + "https://play.google.com" + link)
    print("-" * 120)

제목 : 베놈   Venom
할인 전 가격 : ₩1,800
할인 가격 : ₩900
링크 : https://play.google.com/store/movies/details/%EB%B2%A0%EB%86%88_Venom?id=uEv-FGwTDEw
------------------------------------------------------------------------------------------------------------------------
제목 : 블러드샷
할인 전 가격 : ₩3,500
할인 가격 : ₩1,800
링크 : https://play.google.com/store/movies/details/%EB%B8%94%EB%9F%AC%EB%93%9C%EC%83%B7?id=xu4ldemoFN4.P
------------------------------------------------------------------------------------------------------------------------
제목 : 온워드: 단 하루의 기적
할인 전 가격 : ₩11,000
할인 가격 : ₩4,400
링크 : https://play.google.com/store/movies/details/%EC%98%A8%EC%9B%8C%EB%93%9C_%EB%8B%A8_%ED%95%98%EB%A3%A8%EC%9D%98_%EA%B8%B0%EC%A0%81?id=fCwq2DL1fDY.P
------------------------------------------------------------------------------------------------------------------------
제목 : 콜 오브 와일드
할인 전 가격 : ₩4,500
할인 가격 : ₩2,500
링크 : https://play.google.com/store/movies/details/%EC%BD%9C_%EC%98%A4%EB%B8%8C_%EC%99%80%EC%9D%BC%EB%93%9

In [17]:
# 브라우저 종료
browser.quit()